## Hover Tool

In [ ]:
from bokeh.plotting import figure, output_notebook, show
output_notebook() # a bokeh resource

In [ ]:
from bokeh.models import HoverTool, ColumnDataSource

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
))
hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
])
p = figure(plot_width=600, plot_height=400, tools=[hover],
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)
show(p)

In [ ]:
source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
    imgs=[
        'https://bokeh.pydata.org/static/snake.jpg',
        'https://bokeh.pydata.org/static/snake2.png',
        'https://bokeh.pydata.org/static/snake3D.png',
        'https://bokeh.pydata.org/static/snake4_TheRevenge.png',
        'https://bokeh.pydata.org/static/snakebite.jpg'
    ],
    fonts=[
        '<i>italics</i>',
        '<pre>pre</pre>',
        '<b>bold</b>',
        '<small>small</small>',
        '<del>del</del>'
    ]
))

hover = HoverTool( tooltips="""
    <div>
        <div>
            <img
                src="@imgs" height="42" alt="@imgs" width="42"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@desc</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span>@fonts{safe}</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>
    """
)

p = figure(plot_width=600, plot_height=400, tools=[hover],
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

## Interaction with widgets and JS CallBacks

In [ ]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button

button_warning = Button(label="Foo Warning", button_type="warning")
button_success = Button(label="Foo Success", button_type="success")

show(widgetbox(button_warning, button_success))

In [ ]:
from bokeh.models.widgets import Slider, RangeSlider

slider = Slider(start=0, end=10, value=3, step=.1, title="Slider")
range_slider = RangeSlider(start=0, end=10, value=(5,9), step=.1, title="Range Slider")

show(widgetbox(slider, range_slider))

In [ ]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure

p1 = figure(plot_width=500, plot_height=500)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="red", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=500, plot_height=500)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="blue", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

## CustomJS callbacks


In [ ]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

In [ ]:
from random import random

from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.callback = CustomJS(args=dict(s2=s2), code="""
        var inds = cb_obj.selected.indices;
        var d1 = cb_obj.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)

layout = row(p1, p2)

show(layout)